In [15]:
import os
import librosa
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from scipy.fft import fft, fftfreq
from scipy.signal import spectrogram, find_peaks
from scipy.io import wavfile
import pandas as pd
import torch.nn as nn
import torch
import copy
import numpy as np
import torch.optim as optim
import tqdm

In [16]:
class Deep(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(14, 128)
        self.act1 = nn.ReLU()
        self.layer2 = nn.Linear(128, 32)
        self.act2 = nn.ReLU()
        self.output = nn.Linear(32, 1)
        self.sigmoid = nn.Sigmoid()
        self.dropout = nn.Dropout(0.2)
        

    def forward(self, x):
        x = self.act1(self.layer1(x))
        x = self.act2(self.layer2(x))
        x = self.dropout(x)
        x = self.sigmoid(self.output(x))
        return x

In [17]:
df=pd.read_csv('dataframe.csv')

In [18]:
columns2 = ['Min','Max','Harmonic number 1',
       'Harmonic number 2', 'Harmonic number 3', 'Harmonic number 4',
       'Harmonic number 5', 'Harmonic number 6', 'Harmonic number 7',
       'Harmonic number 8','Int 1','Int 2','Int 3','Int 4']
dftr=df[columns2].to_numpy()
dftry=df["Output"].to_numpy()

In [19]:
def model_train(model, X_train, y_train, X_val, y_val):
    # loss function and optimizer
    loss_fn = nn.BCELoss()  # binary cross entropy
    optimizer = optim.Adam(model.parameters(), lr=0.0001)

    n_epochs = 200   # number of epochs to run
    batch_size = 5  # size of each batch
    batch_start = torch.arange(0, len(X_train), batch_size)

    # Hold the best model
    best_acc = - np.inf   # init to negative infinity
    best_weights = None

    for epoch in range(n_epochs):
        model.train()
        with tqdm.tqdm(batch_start, unit="batch", mininterval=0, disable=True) as bar:
            bar.set_description(f"Epoch {epoch}")
            for start in bar:
                # take a batch
                X_batch = X_train[start:start+batch_size]
                y_batch = y_train[start:start+batch_size]
                # forward pass
                y_pred = model(X_batch)
                loss = loss_fn(y_pred, y_batch)
                # backward pass
                optimizer.zero_grad()
                loss.backward()
                # update weights
                optimizer.step()
                # print progress
                acc = (y_pred.round() == y_batch).float().mean()
                bar.set_postfix(
                    loss=float(loss),
                    acc=float(acc)
                )
        # evaluate accuracy at end of each epoch
        model.eval()
        y_pred = model(X_val)
        acc = (y_pred.round() == y_val).float().mean()
        acc = float(acc)
        if acc > best_acc:
            best_acc = acc
            best_weights = copy.deepcopy(model.state_dict())
    # restore model and return best accuracy
    model.load_state_dict(best_weights)
    return best_acc

In [20]:
xdf = torch.tensor(dftr, dtype=torch.float32)
ydf = torch.tensor(dftry, dtype=torch.float32).reshape(-1, 1)

In [30]:
from sklearn.model_selection import StratifiedKFold, train_test_split
X_train, X_test, y_train, y_test = train_test_split(xdf, ydf, train_size=0.85, shuffle=True)

In [27]:
# define 5-fold cross validation test harness
kfold = StratifiedKFold(n_splits=2, shuffle=True)
cv_scores_wide = []
for train, test in kfold.split(X_train, y_train):
    # create model, train, and get accuracy
    model = Deep()
    acc = model_train(model, X_train[train], y_train[train], X_train[test], y_train[test])
    #print("Accuracy (wide): %.2f" % acc)
    cv_scores_wide.append(acc)

In [31]:
print("Accuracy: %.2f%% (+/- %.2f%%)" % (np.mean(cv_scores_wide)*100, np.std(cv_scores_wide)*100))

Accuracy: 63.01% (+/- 1.10%)
